# Linear optimization under constraints

## 01. Imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime as dt 
import pulp
from itertools import groupby

## 02. Parameters

In [6]:
DAY = pd.DataFrame(
    columns=[
        'Client', 
        'Location', 
        'Task', 
        'Start_min', 
        'End_max', 
        'Duration'
    ]
)

TASKS_CLIENT = pd.DataFrame(
    columns=[
        'Id',
        'Client', 
        'Task', 
        'Duration' # derived from DAY dataframe
    ]
)
# set Id as index
# keep 0 for traveling, 1 for waiting

INTER = pd.DataFrame(columns=['ID Intervenant'])

time_slots = list(range(24*12))

## 03. Variables definition

In [ ]:
def get_distance(prec_client_task, next_client_task):
    return

In [ ]:
# Define PuLP problem
prob = pulp.LpProblem('HYBRID-UK', pulp.LpMaximize)

# Variables are : for each intervenant, a list of actions x client (with a 5 min granularity)
variables_dict = {
    id_inter: {
        k: pulp.LpVariable(
            cat="Integer",
            lowBound=TASKS_CLIENT['Id'].min(),
            upBound=TASKS_CLIENT['Id'].max(),
            name=f"inter_1_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}

is_traveling = {
    id_inter: {
        k: pulp.LpVariable(
            cat="Integer",
            lowBound=0,
            upBound=1,
            name=f"is_traveling_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in is_traveling:
    prob += is_traveling[id_inter] == [int(task_client!=0) for task_client in variables_dict[id_inter]]

is_waiting = {
    id_inter: {
        k: pulp.LpVariable(
            cat="Integer",
            lowBound=0,
            upBound=1,
            name=f"is_waiting_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in is_waiting:
    prob += is_waiting[id_inter] == [int(task_client!=1) for task_client in variables_dict[id_inter]]

is_at_same_client = {
    id_inter: {
        k: pulp.LpVariable(
            cat="Integer",
            lowBound=0,
            upBound=1,
            name=f"is_at_same_client_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in is_waiting:
    prob += is_waiting[id_inter] == [int(task_client!=1) for task_client in variables_dict[id_inter]]


# CONSTRAINTS

for inter, variable in variables_dict.items(): # for each client
    # list of successive similar positions
    grouped_variables = [list(group) for _, group in groupby(list(variable))]
    unique_client_tasks = [var[0] for var in grouped_variables]
    
    # if activity chages between 2 timeslots, the intervenant is either traveling, waiting or doing another activity at the same client
    for i in range(len(variable)-1):
        if variable[i+1] != variable[i]:
            prob += is_traveling[inter][i+1]+is_waiting[inter][i+1]+is_at_same_client[inter][i+1] == 1
        
        # having 0-1-0 is absurd (traveling - waiting - traveling)
        # if there is a wait, we define by default that it shall be after a travel
        if variable[i]==1:
            prob += variable[i+1]>=1
    
    # successive timeslots doing 1 client_task shall match the task duration
    for sublist in grouped_variables:
        client_task_id = sublist[0]
        duration = TASKS_CLIENT.loc[client_task_id]['Duration']
        prob += len(sublist) == duration // 5
    
    # traveling shall match the expected travel duration
    for i, sublist in enumerate(grouped_variables[1:-1]):
        prec_client_task = grouped_variables[i-1][0]
        next_client_task = grouped_variables[i+1][0]
        if sublist[0]==0: # if the intervenant is traveling
            prob += len(sublist) == get_distance(prec_client_task, next_client_task)
    
    
    

In [ ]:
def get_duration(client_task_id):
    return TASKS_CLIENT.loc[client_task_id]['Duration']

In [7]:
dict_1 = {
    0: 1, 
    1: 3, 
    2: 3, 
}
list(dict_1)

[0, 1, 2]

## 04. Optimisation